# Characterization of Alzheimer's Disease through clustering of Medical notes

## Introduction

### Importing packages

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans

### Data Pre-processing

#### Exploring the data

In [2]:
path = "data/Early-onset.csv"
df = pd.read_csv(path)
df

,person_id,birth_datetime,gender_source_value,race_source_value,ethnicity_source_value,note_title,note_text
0,271,1932-02-09 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,No acute osseous abnormality of the right shou...
1,271,1932-02-09 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,FINDINGS OF PULMONARY EDEMA. ACUITY LEVEL: 2. ...
2,271,1932-02-09 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,No acute osseous abnormality of the right shou...
3,271,1932-02-09 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,No acute osseous abnormality of the right shou...
4,271,1932-02-09 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,1. Moderate subcutaneous hematoma about the le...
...,...,...,...,...,...,...,...
26182,258985,1946-12-18 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,X-ray abdomen complete with chest Indication: ...
26183,258985,1946-12-18 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,CTA dissection protocol. Technique: Axial imag...
26184,258985,1946-12-18 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,CTA dissection protocol. Technique: Axial imag...
26185,258985,1946-12-18 00:00:00,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,Study: AP radiograph of the chest History: Wea...


### Calculating the age of each patient from their birth date

In [3]:
# Converting the "birth_datetime" column to a datetime format 
df["birth_datetime"] = pd.to_datetime(df["birth_datetime"], format='%Y-%m-%d')
# Calculating the age by substracting the current date with the date in the dataframe
df["age"] = (pd.Timestamp('now') - df["birth_datetime"]).astype('timedelta64[Y]')
# Converting age to integer
df["age"]  = pd.to_numeric(df["age"], downcast='integer')
# Dropping the birth_datetime (it's useless at this point) 
df = df.drop("birth_datetime", axis=1)
# Re-arranging the order of the columns
df = df[['person_id', 'age', 'gender_source_value', 'race_source_value', 'ethnicity_source_value', 'note_title', 'note_text']]
df

,person_id,age,gender_source_value,race_source_value,ethnicity_source_value,note_title,note_text
0,271,90,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,No acute osseous abnormality of the right shou...
1,271,90,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,FINDINGS OF PULMONARY EDEMA. ACUITY LEVEL: 2. ...
2,271,90,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,No acute osseous abnormality of the right shou...
3,271,90,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,No acute osseous abnormality of the right shou...
4,271,90,FEMALE,WHITE,NOT HISPANIC,order_impression: IMAGING,1. Moderate subcutaneous hematoma about the le...
...,...,...,...,...,...,...,...
26182,258985,75,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,X-ray abdomen complete with chest Indication: ...
26183,258985,75,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,CTA dissection protocol. Technique: Axial imag...
26184,258985,75,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,CTA dissection protocol. Technique: Axial imag...
26185,258985,75,FEMALE,WHITE,NOT HISPANIC,order_narative: IMAGING,Study: AP radiograph of the chest History: Wea...


### Creating the corpus from medical notes 

In [10]:
corpus = list(df["note_text"])

### Creating Text embeddings

In [11]:
model = SentenceTransformer("emilyalsentzer/Bio_ClinicalBERT")

No sentence-transformers model found with name /home/ricardo/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/ricardo/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertFo

#### Some model parameters

The Transformer itself has a max sequence length of 512 tokens and it does not lowercase the input. For the pooling operation we will produce a 768-dimensional sentence embedding. We do this using mean pooling 

In [12]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [13]:
embeddings = model.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/819 [00:00<?, ?it/s]

In [33]:
embeddings

array([ 1.43734232e-01, -2.47471109e-01, -1.05535328e-01, -1.42186433e-01,
        4.32358295e-01, -1.21277474e-01,  1.45889536e-01,  9.81251895e-02,
       -3.80362898e-01, -8.11375603e-02,  1.63590107e-02,  2.73184448e-01,
        1.46680295e-01, -3.32147665e-02, -3.92429605e-02,  1.45066589e-01,
        2.84236580e-01, -2.43330061e-01, -2.14798614e-01,  3.79987657e-01,
       -7.70591479e-03, -1.06782475e-02, -2.52945006e-01, -4.26073521e-01,
        2.45417461e-01, -3.58902842e-01, -1.26812458e-01,  6.37156427e-01,
        9.68703032e-02,  2.84381807e-01, -5.73599711e-02,  5.57643473e-02,
       -1.03775961e-02, -1.05977982e-01, -8.02437961e-02,  2.32724786e-01,
        4.55117732e-01,  4.13280010e-01, -9.34125390e-03,  4.33911830e-01,
        4.83619571e-01, -2.79802829e-01,  2.82918900e-01,  4.77648616e-01,
        8.67762864e-02, -4.74859178e-01,  7.42306113e-02, -1.47349238e-01,
       -5.07345438e-01,  2.15872571e-01,  2.80527025e-01,  1.66475609e-01,
       -2.06533462e-01, -